In [1]:
import pandas as pd
import pyodbc 
import sqlalchemy as sq
import datetime
pd.set_option('display.max_columns', None)
import os
import time
import datetime
from zipfile import ZipFile

In [2]:
from AllFunctions.sendemail import sendemailwithattachment
from AllFunctions.scanemail import main
from AllFunctions.runstoreproc import runstoreprocedure
from AllFunctions.runsql import runcommand
import random

In [3]:
def runstoreprocedure(con,sp):
    cursor=con.cursor()
    cursor.execute(sp)
    cursor.commit()
    cursor.close()
    con.close()

In [4]:
def exportsqldata(command,DB,serv):
    con=pyodbc.connect('Trusted_Connection=yes',driver ='{SQL Server}',server=serv, database =DB)
    cursor=con.cursor()
    cursor.execute(command)
    l=[]
    while 1:
        head =cursor.fetchone()
        if not head:
            break
        [l.append(s) for s in head]
    return l

In [5]:
def exportdata(con,command):
    data=pd.read_sql(command,con)
    return data

In [6]:
def importsqlcommands(filename):
    csvfilescreated=[]
    with open (filename,'r') as fd:
        sqlfile=fd.read()
    print(sqlfile)
    return sqlfile

In [7]:
def createdb():
    #create database if it does not exist
    #if exist, open SQL server to manually drop the database
    serv ='NMALI-T450S'
    DB ='master'
    con=pyodbc.connect('Trusted_Connection=yes',driver ='{SQL Server}',server=serv, database =DB)
    #drop database if exist
    command="SELECT name FROM master.dbo.sysdatabases WHERE ('[' + name + ']' = 'event' OR name = 'event')"
    filename ="D:\Jupyter Notebook\AK_Project2\sql_scripts\Drop data base.sql"
    df=exportsqldata(command,DB,serv)
    if df:
        print('Please drop the database before you create a new one')
        con.autocommit
        cursor=con.cursor()
        cursor.execute(importsqlcommands(filename))
        cursor.commit()
        cursor.close()
        runstoreprocedure(con,'exec create_database')

    else:
        runstoreprocedure(con,'exec create_database')
    
    if df:
        return True
    else:
        return False

In [8]:
def createstoreproc():
    #call Master scripts to create store procedures
    serv ='NMALI-T450S'
    DB ='Event'
    con=pyodbc.connect('Trusted_Connection=yes',driver ='{SQL Server}',server=serv, database =DB)
    #Location of master script
    filename ="D:\Jupyter Notebook\AK_Project2\sql_scripts\Store_Procedure_Creation\Master_Script.sql"
    cursor=con.cursor()
    cursor.execute(importsqlcommands(filename))
    cursor.commit()
    cursor.close()
    con.close()
        

In [9]:
def insertdata():
    #call Master scripts to run and insert the data in the database
    x1=time.time()
    serv ='NMALI-T450S'
    DB ='Event'
    #Location of master script
    filename ="D:\Jupyter Notebook\AK_Project2\sql_scripts\SQL_Independent_Scripts\Master_Script.sql"
    file=importsqlcommands(filename)
    for each in file.split(';'):
        con=pyodbc.connect('Trusted_Connection=yes',driver ='{SQL Server}',server=serv, database =DB)
        cursor=con.cursor()
        cursor.execute(each)
        cursor.commit()
        cursor.close()
        con.close()
    x2=time.time()
    diff=str((x2-x1)//60)+" Minutes"     
    print('Program took ',diff,' to load and create index on the dataset')
    
    

In [10]:

def dataprocess():
    #call procssing scripts to run and process the data in the database
    x1=time.time()
    serv ='NMALI-T450S'
    DB ='Event'
    #Location of master script
    filename ="D:\Jupyter Notebook\AK_Project2\sql_scripts\Data_processing.sql"
    file=importsqlcommands(filename)
    for each in file.split(';'):
        con=pyodbc.connect('Trusted_Connection=yes',driver ='{SQL Server}',server=serv, database =DB)
        cursor=con.cursor()
        cursor.execute(each)
    cursor.commit()
    cursor.close()
    con.close()
    x2=time.time()
    diff=str((x2-x1)//60)+" Minutes"     
    print('Program took ',diff,' to load and create index on the dataset')
    

In [11]:
date = (datetime.datetime.now().strftime("%d-%b-%Y"))
print(date)
username = 'nitsmali@hotmail.com'
password = 'mylifemyruleS9'
downloadattach=False
numberofemails=3

currentdate=datetime.datetime.now().strftime("%Y%m%d")
allocatedtime=datetime.datetime.now().strftime("%H")
print(allocatedtime)

listemail =['nitsmali@hotmail.com']
flag=True
while allocatedtime>='12' and allocatedtime<='23' and flag==True:
    searchfrom=random.choice(listemail)
    allocatedtime=datetime.datetime.now().strftime("%H")
    print(allocatedtime)
    if allocatedtime>='16':
        datastamp=main(username,password,searchfrom,downloadattach,date,numberofemails)
        print(datastamp)
        if datastamp:
            for key,value in datastamp.items():
                if  (value['Body'].lower().find('posted=yes')!=-1):
                    flag=False
                    sendemailwithattachment(username,password,"nitsmali@hotmail.com","nitsmali@hotmail.com","","Data Loading has been started","Hi Team,\nAutomated Data Loading has been started.\n\nThank You\nNitin",[])
                    print('Data Loading has been started')
                    time.sleep(30)
                    #createdb()
                    #createstoreproc()
                    #insertdata()
                    sendemailwithattachment(username,password,"nitsmali@hotmail.com","nitsmali@hotmail.com","","Data Loading Completed","Hi Team,\nAutomated Data Loading has been completed.\n Data processing has been started.\n\nThank You\nNitin",[])
                    print('Data process started')
                    #dataprocess()
                    time.sleep(30)
                    sendemailwithattachment(username,password,"nitsmali@hotmail.com","nitsmali@hotmail.com","","Data Processing Completed","Hi Team,\nAutomated Data Processing has been completed.\n\nThank You\nNitin",[])
                    
    time.sleep(30)

02-Sep-2019
22
22
{'21:09:29': {'Time': '21:09:29', 'From': 'nitsmali@hotmail.com', 'To': 'Nitin Mali <nitsmali@hotmail.com>, "sys.ta.services@klarna.com"\r\n\t<sys.ta.services@klarna.com>', 'Subject': 'RE: Reminder: Test request from Klarna', 'Body': 'Hello Kattis,  I have completed the test. Thank you.  Best regards, Nitin Mali Data Scientist/Analyst mailto:Â\xa0nitsmali@hotmail.com phone: 908-275-7808   -----Original Message----- From: Nitin Mali Sent: Thursday, August 29, 2019 7:30 PM To: sys.ta.services@klarna.com Subject: RE: Reminder: Test request from Klarna  Hello Kattis,  Yes. I will answer the test as soon as possible. Thank you.  Best regards, Nitin Mali Data Scientist/Analyst mailto:Â\xa0nitsmali@hotmail.com phone: 908-275-7808   -----Original Message----- From: Kattis Sent: Thursday, August 29, 2019 7:15 PM To: Nitin Mali Subject: Reminder: Test request from Klarna  Hello Nitin Mali,  3 days ago, we sent you an e-mail requesting that you take a test. You will find the tes